In [1]:
on_kaggle = True

In [2]:
OTHRESH=0.05    
PTHRESH=0.94   
AVPTHRESH=0.65  
THRESH = .175
MIN_MAX_MRCNN_CONF = .975
MINPROB = 0.69  
C1 = 1e6  
C2 = 0.06
UNET_MINPROB = 0.28  
UNET_MINCONF = 0.35  
YOLO_MINCONF = 0.15  
MINMIN = 0.20  

In [3]:
import numpy as np 
import pandas as pd 
import os
from sklearn import metrics
from scipy.special import logit,expit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error

In [4]:
os.listdir('../input')

['rsna-pneumonia-detection-challenge',
 'rsna-oof-predictions',
 'combine-fold-class-predictions-stage-2',
 'yolov3-inference-from-more-saved-weights-stage-2',
 'mrcnn-inference-from-saved-wts-4dig-70-stage-2',
 'filter-199-final-with-higher-thresh-stage-2',
 'gs-dense-chexnet-predict-stage-2-from-all-models',
 'yolov3-inference-from-multiple-saved-wts-stage-2',
 'mrcnn-inference-from-saved-wts-2dig-95-stage-2',
 'andy-densenet-from-multiple-saved-weights-stage-2',
 'resnet-unet-from-saved-weights-stage-2']

In [5]:
!ls ../input/gs-dense-chexnet-predict-test-from-all-models

ls: cannot access ../input/gs-dense-chexnet-predict-test-from-all-models: No such file or directory


In [6]:
RESULTS_LOC = '../input'  # Outputs from base models
RAW_DATA_LOC = '../input/rsna-pneumonia-detection-challenge'          # Raw input data
YOLO_SUB_STEM = 'yolov3-inference-from-multiple-saved-wts-stage-2/submission_yolo'    # YoloV3 predictions
YOLO_SUB_STEM2 = 'yolov3-inference-from-more-saved-weights-stage-2/submission_yolo'    # YoloV3 predictions
MORE_YOLO_EPOCHS = [5300, 1500, 6700, 2000] if on_kaggle else []
UNET_RESULTS_PATH = '../input/resnet-unet-from-saved-weights-stage-2/submission_resnet34unet_thr0.35.csv'  # Resnet-Unet predictions
CHEXNET_OOF_STEM = 'rsna-oof-predictions/predictions_valid_fold_'        # Giulia's classification model
CHEXNET_TEST_STEM = 'gs-dense-chexnet-predict-stage-2-from-all-models/test_preds_pth_fold'
SPECIAL_FILE0 = 'gs-dense-chexnet-predict-stage-2-from-all-models/test_preds_pth_fold0_for_combined_folds'
CHEXNET_TEST_FILE0 = SPECIAL_FILE0 if on_kaggle else CHEXNET_TEST_STEM + '0'
ADENSE_TEST_STEM = 'andy-densenet-from-multiple-saved-weights-stage-2/submission_adense_f'                    # Andy's classification model
ADEMSE_OOF_STEM = 'rsna-oof-predictions/val_dense_v5_a1_f'
MRCNN_TEST_STEM = 'mrcnn-inference-from-saved-wts-4dig-70-stage-2/submission_mrcnn_v'  # MRCNN predicted boxes
MRCNN95_TEST_STEM = 'mrcnn-inference-from-saved-wts-2dig-95-stage-2/submission_mrcnn_v' \
   if on_kaggle else MRCNN_TEST_STEM  # MRCNN predicted boxes
MRCNN_OUT_STEM = 'temp_mrcnn_v' if on_kaggle else MRCNN_TEST_STEM
MRCNN_OOF_STEM = 'rsna-oof-predictions/val_v'
MRCNN_OOF_OUT_STEM = 'val_v' if on_kaggle else MRCNN_OOF_STEM
MRCNN_TEST_SEP = ''
MRCNN_OOF_SEP = '_'
IN_OOF_SUFFIX = ''
IN_SUFFIX = '.csv'
OUT_SUFFIX = '.csv'
CHEX_CLASS_PRED_PATH = '../input/combine-fold-class-predictions-stage-2/test_probs.csv'  # Logisitc average of Giulia's test class preds
KERNEL_OUTPUT_PATH = '../input/filter-199-final-with-higher-thresh-stage-2/filter199.csv'      # "Pneumonia - Segm. filtered through Class." kernel

In [7]:
N_FOLDS = 5
OUTPUT_LOC = '.'

OOF_INFILE_STEM = MRCNN_OOF_STEM + '9' + MRCNN_OOF_SEP + 'a1' + MRCNN_OOF_SEP + 'f'
vers = '8' if on_kaggle else '9'
TEST_INFILE_STEM = MRCNN_TEST_STEM + vers + MRCNN_TEST_SEP + 'a1' + MRCNN_TEST_SEP + 'f'
OOF_OUTFILE_NAME = MRCNN_OOF_OUT_STEM + '10' + MRCNN_OOF_SEP + 'a1' + \
                   MRCNN_OOF_SEP + 'allfolds_out.csv'

TEST_OUTFILE_STEM = MRCNN_OUT_STEM + '10' + MRCNN_TEST_SEP + 'a1' + MRCNN_TEST_SEP + 'f_out'
OOF_CLASS_PROBS_OUTFILE = 'phase3_oof_out.csv'
TEST_CLASS_PROBS_OUTFILE_STEM = 'phase3_test_out'
FULL_TEST_CLASS_PROBS_OUTFILE = 'phase3_test_avg_out.csv'

In [8]:
dfs = []

for a in range(1,3):
    for f in range(5):
        vers = '8' if a==2 and f==0 else '1'
        fn = RESULTS_LOC +'/'+ MRCNN95_TEST_STEM + vers + MRCNN_TEST_SEP + \
             'a' + str(a) + MRCNN_TEST_SEP + 'f' + str(f) + IN_SUFFIX
        dfs.append( pd.read_csv(fn).set_index('patientId') )

for i,f in enumerate(dfs):
    if i:
        df = df.join(f.rename(columns={'PredictionString':'pred'+str(i)}))
    else:
        df = f.rename(columns={'PredictionString':'pred'+str(i)})

In [9]:
def non_max_suppression(boxes, probs=None, overlapThresh=OTHRESH):
	if len(boxes) == 0:
		return []
	if boxes.dtype.kind == "i":
		boxes = boxes.astype("float")

	pick = []

	x1 = boxes[:, 0]
	y1 = boxes[:, 1]
	x2 = boxes[:, 2]
	y2 = boxes[:, 3]

	area = (x2 - x1 + 1) * (y2 - y1 + 1)
	idxs = y2

	if probs is not None:
		idxs = probs

	idxs = np.argsort(idxs)

	while len(idxs) > 0:
		last = len(idxs) - 1
		i = idxs[last]
		pick.append(i)

		xx1 = np.maximum(x1[i], x1[idxs[:last]])
		yy1 = np.maximum(y1[i], y1[idxs[:last]])
		xx2 = np.minimum(x2[i], x2[idxs[:last]])
		yy2 = np.minimum(y2[i], y2[idxs[:last]])

		w = np.maximum(0, xx2 - xx1 + 1)
		h = np.maximum(0, yy2 - yy1 + 1)

		overlap = (w * h) / area[idxs[:last]]

		idxs = np.delete(idxs, np.concatenate(([last],
			np.where(overlap > overlapThresh)[0])))

	return boxes[pick].astype("int"), list(np.array(probs)[pick])

In [10]:
box_dict = {}
conf_dict = {}
for pat,row in df.iterrows():
    boxes = []
    confs = []
    maxconfs = []
    for pred in row:
        maxconf = 0.
        if isinstance(pred, str):
            s = pred.split(' ')
            if s[-1]=='':
                s.pop()  
            if s[0]=='':
                s.pop(0)  
            if ( len(s)%5 ):
                print( 'Bad prediction string.')
            while len(s):
                conf = float(s.pop(0))
                x = int(round(float(s.pop(0))))
                y = int(round(float(s.pop(0))))
                w = int(round(float(s.pop(0))))
                h = int(round(float(s.pop(0))))
                if conf>maxconf:
                    maxconf = conf
                if conf>PTHRESH:
                    boxes.append( [x,y,x+w,y+h] )
                    confs.append( conf )
        maxconfs.append(maxconf)
    avgconf = sum(maxconfs)/len(maxconfs)
    if len(boxes) and avgconf>AVPTHRESH:
        box_dict[pat] = boxes
        conf_dict[pat] = confs
len(box_dict), len(conf_dict)

(1273, 1273)

In [11]:
box_dict_nms = {}
conf_dict_nms = {}
for p in box_dict:
    boxes, confs = non_max_suppression(np.array(box_dict[p]), np.array(conf_dict[p]))
    box_dict_nms[p] = boxes
    conf_dict_nms[p] = confs

In [12]:
sub_dict = {}
for p in df.index:
    predictionString = ''
    if p in box_dict_nms:
        for box, conf in zip(box_dict_nms[p], conf_dict_nms[p]):
            x, y, x2, y2 = box
            height = y2 - y
            width = x2 - x
            predictionString += str(conf) + ' ' + str(x) + ' ' + str(y) + ' ' + str(width) + ' ' + str(height) + ' '
    sub_dict[p] = predictionString

sub = pd.DataFrame.from_dict(sub_dict,orient='index')
sub.index.names = ['patientId']
sub.columns = ['PredictionString']
phase0_output = sub

In [13]:
probs = pd.read_csv(CHEX_CLASS_PRED_PATH)

In [14]:
df = phase0_output.join(probs.set_index('patientId'))
out = df.copy()
out.loc[df.prob<THRESH,'PredictionString'] = np.nan
out.loc[df.PredictionString=='','PredictionString'] = np.nan
phase1_output = out.drop(['prob'],axis=1)

In [15]:
df1 = pd.read_csv(KERNEL_OUTPUT_PATH).set_index('patientId')
df2 = phase1_output
df1_cases = df1[~df1.PredictionString.isnull()]
df2_cases = df2[~df2.PredictionString.isnull()]
df1_pos_ids = df1_cases.index.values
df2_pos_ids = df2_cases.index.values

In [16]:
df1_only_dict = {}
for p in df1_pos_ids:
    if not p in df2_pos_ids:
        df1_only_dict[p] = float(df1_cases.loc[p,'PredictionString'].split(' ')[0])

In [17]:
df2_only_dict = {}
for p in df2_pos_ids:
    if not p in df1_pos_ids:
        df2_only_dict[p] = float(df2_cases.loc[p,'PredictionString'].split(' ')[0])

In [18]:
df_out = df1.copy()
for p,r in df1.iterrows():
    if p in df2_only_dict:
        if df2_only_dict[p] > MIN_MAX_MRCNN_CONF:
            df_out.loc[p,'PredictionString'] = df2.loc[p,'PredictionString']
phase2_output = df_out

In [19]:
phase2_output.head()

,PredictionString
patientId,
c103f115-266b-4f0c-97d0-082dc2438a27,0.97 612 372 268 416 0.97 240 568 188 184
2ba3df76-d959-414f-b466-4bc0b4f41135,0.98 188 252 224 252 0.95 588 240 228 340
1114eeb8-c035-42f4-a038-13a806a568ad,NaN
2276b258-5b47-4c7d-ab1f-5b2bae4a8fc4,NaN
23ffe96d-ef6c-4a8f-914e-12754fe689c4,NaN


In [20]:
input_dir = RESULTS_LOC
infile_stem = OOF_INFILE_STEM
oof_preds_input = []
for ifold in range( N_FOLDS ):
    fp_in = os.path.join(input_dir, infile_stem + str(ifold) + IN_OOF_SUFFIX)
    oof_preds_input.append( pd.read_csv(fp_in) )
    oof_preds_input[-1]['fold'] = ifold
oof_preds = pd.concat(oof_preds_input).set_index('patientId')

In [21]:
probs_dict = {}
for p, r in oof_preds.iterrows():
    probs_dict[p] = MINPROB
oof_positive = oof_preds[~oof_preds.PredictionString.isnull()]
for p, r in oof_positive.iterrows():
    s = r.PredictionString.split(' ')
    if s[-1]=='':
        s.pop()  
    if s[0]=='':
        s.pop(0)  
    if ( len(s)%5 ):
        print( 'Bad prediction string.')
    while len(s):
        conf = float(s.pop(0))
        x = int(round(float(s.pop(0))))
        y = int(round(float(s.pop(0))))
        w = int(round(float(s.pop(0))))
        h = int(round(float(s.pop(0))))
        if conf>probs_dict[p]:
            probs_dict[p] = conf
prob_df = pd.DataFrame(probs_dict,index=['prob']).transpose()

In [22]:
input_dir = RAW_DATA_LOC
fp = os.path.join(input_dir, 'stage_2_train_labels.csv')
act = pd.read_csv(fp).set_index('patientId').rename(columns={'Target':'actual'})[['actual']]
act = act[~act.index.duplicated()]

In [23]:
df = act.join(prob_df,how='right')  
df.head()

,actual,prob
ea0f76a2-08d3-4273-b9c3-1253483fa647,0,0.9259
e3cacb25-a167-4e17-b869-dcabef2f2baa,0,0.9683
63396b9b-b228-4fe6-8aeb-5cd4862c15f5,0,0.9030
dae4e147-89d1-4110-bbeb-90ac76ede50a,1,0.9594
a7a95de8-1075-49df-a0ed-edc1baca084d,1,0.9807


In [24]:
rawprobs = df.prob.values.reshape(-1,1)
lr = LogisticRegression(C=C1)
lr.fit(logit(rawprobs),df.actual)
b1 = lr.coef_[0,0]
b0 = lr.intercept_[0]
b0, b1

(-5.812614116759858, 1.488076663905564)

In [25]:
folds = []
for ifold in range(N_FOLDS):
    folds.append( oof_preds[oof_preds.fold==ifold].index.values )

In [26]:
def run_logistic_by_fold():
    df = act.join(prob_df,how='right')   
    logits = df.prob.apply(logit)
    df['oofprob'] = np.nan
    b1 = []
    b0 = []
    for i in range(len(folds)):

        fs = folds.copy()
        te = fs.pop(i)  
        tr = np.concatenate(fs)  

        Xtr = logits[tr].values.reshape(-1,1)
        Xte = logits[te].values.reshape(-1,1)
        ytr = df.actual[tr].copy()
        yte = df.actual[te].copy()

        lr.fit(Xtr, ytr)
        df.loc[te,'oofprob'] = lr.predict_proba(Xte)[:,1]
        b1.append(lr.coef_[0,0])
        b0.append(lr.intercept_[0])

    coefs = pd.DataFrame( {'b0':b0, 'b1':b1}, index=range(len(folds)) )
    coefs.index.name = 'fold'

    return( df.sort_values('oofprob'), coefs )   

In [27]:
lr = LogisticRegression(C=C2)

In [28]:
df, coefs = run_logistic_by_fold()

In [29]:
pat_dict = {}
fold_dict = {}
for pat,row in oof_preds.iterrows():
    fold = row.fold
    pred = row.PredictionString
    boxes = []
    confs = []
    if isinstance(pred, str):
        s = pred.split(' ')
        if s[-1]=='':
            s.pop()  
        if s[0]=='':
            s.pop(0)  
        if ( len(s)%5 ):
            print( 'Bad prediction string.')
        b0 = coefs.b0[fold]
        b1 = coefs.b1[fold]
        while len(s):
            conf = float(s.pop(0))
            x = int(round(float(s.pop(0))))
            y = int(round(float(s.pop(0))))
            w = int(round(float(s.pop(0))))
            h = int(round(float(s.pop(0))))
            boxes.append( [x,y,w,h] )
            confs.append( expit( b0+b1*logit(conf) ) )
    predictionString = ''
    if len(boxes):
        for box, conf in zip(boxes, confs):
            x, y, w, h = box
            predictionString += '{:6.4f} '.format(conf)
            predictionString += str(x) + ' ' + str(y) + ' ' + str(w) + ' ' + str(h) + ' '
    pat_dict[pat] = predictionString
    fold_dict[pat] = fold
xform_oof_preds = pd.DataFrame(pat_dict, 
                               index=['PredictionString']).transpose()
xform_oof_preds.index.name = 'patientId'
xform_oof_preds['fold'] = [fold_dict[x] for x in xform_oof_preds.index.values]
xform_oof_preds.head()

,PredictionString,fold
patientId,,
ea0f76a2-08d3-4273-b9c3-1253483fa647,0.1321 280 432 152 192 0.0525 628 496 200 192,0
e3cacb25-a167-4e17-b869-dcabef2f2baa,0.3108 188 500 216 268 0.0329 592 496 236 304,0
63396b9b-b228-4fe6-8aeb-5cd4862c15f5,0.0962 576 460 292 308 0.0461 224 500 168 216,0
dae4e147-89d1-4110-bbeb-90ac76ede50a,0.2482 568 232 224 556 0.1095 264 548 144 212,0
a7a95de8-1075-49df-a0ed-edc1baca084d,0.4556 640 384 224 284 0.0809 316 412 188 212,0


In [30]:
output_dir = OUTPUT_LOC
xform_oof_preds.to_csv(os.path.join(output_dir, OOF_OUTFILE_NAME))

In [31]:
input_dir = RESULTS_LOC
output_dir = OUTPUT_LOC
infile_stem = TEST_INFILE_STEM
outfile_stem = TEST_OUTFILE_STEM

In [32]:
for ifold in range(N_FOLDS):
    
    fp_in = os.path.join(input_dir, infile_stem + str(ifold) + IN_SUFFIX)
    fp_out = os.path.join(output_dir, outfile_stem + str(ifold) + OUT_SUFFIX)
    inpreds = pd.read_csv(fp_in).set_index('patientId')
    pat_dict = {}

    for pat,row in inpreds.iterrows():
        pred = row.PredictionString
        boxes = []
        confs = []
        if isinstance(pred, str):
            s = pred.split(' ')
            if s[-1]=='':
                s.pop()  # remove terminating null
            if s[0]=='':
                s.pop(0)  # remove initial null
            if ( len(s)%5 ):
                print( 'Bad prediction string.')
            while len(s):
                conf = float(s.pop(0))
                x = int(round(float(s.pop(0))))
                y = int(round(float(s.pop(0))))
                w = int(round(float(s.pop(0))))
                h = int(round(float(s.pop(0))))
                boxes.append( [x,y,w,h] )
                confs.append( expit( b0+b1*logit(conf) ) )
        predictionString = ''
        if len(boxes):
            for box, conf in zip(boxes, confs):
                x, y, w, h = box
                predictionString += '{:6.4f} '.format(conf)
                predictionString += str(x) + ' ' + str(y) + ' ' + str(w) + ' ' + str(h) + ' '
        pat_dict[pat] = predictionString
    outpreds = pd.DataFrame(pat_dict, 
                            index=['PredictionString']).transpose()
    outpreds.index.name = 'patientId'
    outpreds.to_csv(fp_out)
fp_out

'./temp_mrcnn_v10a1f_out4.csv'

In [33]:
outprobs = prob_df.join(oof_preds[['fold']])
b0_ = coefs.b0[outprobs.fold].values
b1_ = coefs.b1[outprobs.fold].values
outprobs['prob'] =  expit( b0_ + logit(outprobs.prob)*b1_ )
outprobs.index.name = 'patientId'
output_dir = OUTPUT_LOC
outprobs.to_csv(os.path.join(output_dir, OOF_CLASS_PROBS_OUTFILE))
outprobs.head()

,prob,fold
patientId,,
ea0f76a2-08d3-4273-b9c3-1253483fa647,0.132091,0
e3cacb25-a167-4e17-b869-dcabef2f2baa,0.310763,0
63396b9b-b228-4fe6-8aeb-5cd4862c15f5,0.096197,0
dae4e147-89d1-4110-bbeb-90ac76ede50a,0.248165,0
a7a95de8-1075-49df-a0ed-edc1baca084d,0.455566,0


In [34]:
output_dir = OUTPUT_LOC
box_outfile_stem = TEST_OUTFILE_STEM
probs_outfile_stem = TEST_CLASS_PROBS_OUTFILE_STEM
minprob_out = expit( b0 + b1*logit(MINPROB))

for ifold in range(N_FOLDS):
    
    fp_in = os.path.join(output_dir, box_outfile_stem + str(ifold) + OUT_SUFFIX)
    inpreds = pd.read_csv(fp_in).set_index('patientId')
    fp_out = os.path.join(output_dir, probs_outfile_stem + str(ifold) + OUT_SUFFIX)

    probs_dict = {}

    for p, r in inpreds.iterrows():
        probs_dict[p] = minprob_out
    inpreds_positive = inpreds[~inpreds.PredictionString.isnull()]
    for p, r in inpreds_positive.iterrows():
        s = r.PredictionString.split(' ')
        if s[-1]=='':
            s.pop()  
        if s[0]=='':
            s.pop(0)  
        if ( len(s)%5 ):
            print( 'Bad prediction string.')
        while len(s):
            conf = float(s.pop(0))
            s.pop(0)
            s.pop(0)
            s.pop(0)
            s.pop(0)
            if conf>probs_dict[p]:
                probs_dict[p] = conf
    outprob_df = pd.DataFrame(probs_dict,index=['prob']).transpose()
    outprob_df.index.name = 'patientId'
    outprob_df.to_csv(fp_out)
    if not ifold:
        allprobs_df = outprob_df.rename(columns={'prob':ifold})
    else:
        allprobs_df = allprobs_df.join(outprob_df.rename(columns={'prob':ifold}))

In [35]:
fp = OOF_CLASS_PROBS_OUTFILE
oof_probs_mrcnn = pd.read_csv(fp).set_index('patientId').rename(columns={'prob':'ph'})

In [36]:
for i in range(N_FOLDS):
    fp = TEST_CLASS_PROBS_OUTFILE_STEM + str(i) + '.csv'
    indf = pd.read_csv(fp).set_index('patientId').rename(columns={'prob':'ph'+str(i)})
    if i:
        test_prob_df = test_prob_df.join(indf)
    else:
        test_prob_df = indf

In [37]:
test_prob_df.head()

,ph0,ph1,ph2,ph3,ph4
patientId,,,,,
30451f0f-6692-4633-b234-94ae2b5960de,0.3098,0.3925,0.232100,0.2020,0.3391
28debac6-3823-4e02-bc83-3fbb9ded1718,0.3363,0.4108,0.229700,0.3239,0.4751
c0ec3097-a14b-4ee2-9547-c0d2532f2e62,0.2473,0.4535,0.216700,0.2501,0.3050
2e5ab719-7447-4d3f-95f8-5ed5c8c736a1,0.2704,0.4160,0.216700,0.4212,0.4334
1d0ce799-81bb-40be-8e92-4b1783906cc9,0.2484,0.0748,0.019257,0.0867,0.1869


In [38]:
f0 = pd.read_csv(RESULTS_LOC + '/' + ADEMSE_OOF_STEM + '0.csv').set_index('patientId')
f1 = pd.read_csv(RESULTS_LOC + '/' + ADEMSE_OOF_STEM + '1.csv').set_index('patientId')
f2 = pd.read_csv(RESULTS_LOC + '/' + ADEMSE_OOF_STEM + '2.csv').set_index('patientId')
f3 = pd.read_csv(RESULTS_LOC + '/' + ADEMSE_OOF_STEM + '3.csv').set_index('patientId')
f4 = pd.read_csv(RESULTS_LOC + '/' + ADEMSE_OOF_STEM + '4.csv').set_index('patientId')

In [39]:
den = pd.concat([f0,f1,f2,f3,f4],axis=0)
print( den.shape )
den.head()

(25684, 2)


,actual,predicted
patientId,,
00436515-870c-4b36-a041-de91049b9ab4,1,0.362432
00704310-78a8-4b38-8475-49f4573b2dbb,1,0.400114
00aecb01-a116-45a2-956c-08d2fa55433f,1,0.777987
01027bc3-dc40-4165-a6c3-d6be2cb7ca34,0,0.005689
6bde463a-d52c-4730-9833-091fc1fef528,0,0.002396


In [40]:
tf0 = pd.read_csv(RESULTS_LOC + '/' + ADENSE_TEST_STEM + '0.csv').set_index('patientId')
tf1 = pd.read_csv(RESULTS_LOC + '/' + ADENSE_TEST_STEM + '1.csv').set_index('patientId')
tf2 = pd.read_csv(RESULTS_LOC + '/' + ADENSE_TEST_STEM + '2.csv').set_index('patientId')
tf3 = pd.read_csv(RESULTS_LOC + '/' + ADENSE_TEST_STEM + '3.csv').set_index('patientId')
tf4 = pd.read_csv(RESULTS_LOC + '/' + ADENSE_TEST_STEM + '4.csv').set_index('patientId')

In [41]:
test_den = tf0.rename(columns={'predicted':'pa0'})
test_den = test_den.join(tf1.rename(columns={'predicted':'pa1'}))
test_den = test_den.join(tf2.rename(columns={'predicted':'pa2'}))
test_den = test_den.join(tf3.rename(columns={'predicted':'pa3'}))
test_den = test_den.join(tf4.rename(columns={'predicted':'pa4'}))
print( test_den.shape )
test_den.head()

(3000, 5)


,pa0,pa1,pa2,pa3,pa4
patientId,,,,,
2d5d8ecc-3ee3-4c9b-bb86-3f614a079585,0.032077,0.065031,0.147478,0.099838,0.112234
2815b7e6-1112-484b-b6ed-367c87e0e22a,0.254421,0.425327,0.037688,0.138590,0.111626
2d17ce84-86f9-4153-8d91-3a75ffe303cc,0.589568,0.423861,0.510739,0.614366,0.228921
0300003e-3405-43a3-b5d9-d314d4810fdb,0.009171,0.007782,0.022904,0.031646,0.021034
21a6488b-80dc-4caf-8d7b-a375011d70dc,0.315024,0.360514,0.273488,0.254714,0.111565


In [42]:
p0 = f0.index.values
p1 = f1.index.values
p2 = f2.index.values
p3 = f3.index.values
p4 = f4.index.values
pt = test_den.index.values

In [43]:
c0 = pd.read_csv(RESULTS_LOC + '/' + CHEXNET_OOF_STEM + '0.csv').set_index('patientId')
c1 = pd.read_csv(RESULTS_LOC + '/' + CHEXNET_OOF_STEM + '1.csv').set_index('patientId')
c2 = pd.read_csv(RESULTS_LOC + '/' + CHEXNET_OOF_STEM + '2.csv').set_index('patientId')
c3 = pd.read_csv(RESULTS_LOC + '/' + CHEXNET_OOF_STEM + '3.csv').set_index('patientId')
c4 = pd.read_csv(RESULTS_LOC + '/' + CHEXNET_OOF_STEM + '4.csv').set_index('patientId')
chex = pd.concat([c0,c1,c2,c3,c4],axis=0)
print( chex.shape )
chex.head()

(25684, 5)


,Unnamed: 0,Target,fold,validPredProba,validPred
patientId,,,,,
00436515-870c-4b36-a041-de91049b9ab4,4,1,0,0.463192,0
00704310-78a8-4b38-8475-49f4573b2dbb,8,1,0,0.679318,1
00aecb01-a116-45a2-956c-08d2fa55433f,14,1,0,0.370484,0
01027bc3-dc40-4165-a6c3-d6be2cb7ca34,23,0,0,0.004191,0
014b7b58-f641-4477-8bbc-ae6f337745d6,29,0,0,0.038145,0


In [44]:
df = den.join(chex[['validPredProba']]).rename(columns={'predicted':'pa','validPredProba':'pg'})
df = df.join(oof_probs_mrcnn).drop(['fold'],axis=1)
print( df.shape )
df.head()

(25684, 4)


,actual,pa,pg,ph
patientId,,,,
00436515-870c-4b36-a041-de91049b9ab4,1,0.362432,0.463192,0.211496
00704310-78a8-4b38-8475-49f4573b2dbb,1,0.400114,0.679318,0.130087
00aecb01-a116-45a2-956c-08d2fa55433f,1,0.777987,0.370484,0.129299
01027bc3-dc40-4165-a6c3-d6be2cb7ca34,0,0.005689,0.004191,0.037949
6bde463a-d52c-4730-9833-091fc1fef528,0,0.002396,0.005397,0.045998


In [45]:
tc0 = pd.read_csv(RESULTS_LOC + '/' + CHEXNET_TEST_FILE0 + '.csv').set_index('patientId')
tc1 = pd.read_csv(RESULTS_LOC + '/' + CHEXNET_TEST_STEM + '1.csv').set_index('patientId')
tc2 = pd.read_csv(RESULTS_LOC + '/' + CHEXNET_TEST_STEM + '2.csv').set_index('patientId')
tc3 = pd.read_csv(RESULTS_LOC + '/' + CHEXNET_TEST_STEM + '3.csv').set_index('patientId')
tc4 = pd.read_csv(RESULTS_LOC + '/' + CHEXNET_TEST_STEM + '4.csv').set_index('patientId')
tc0.head()

,targetPred,targetPredProba
patientId,,
0000a175-0e68-4ca4-b1af-167204a7e0bc,0,0.000299
0005d3cc-3c3f-40b9-93c3-46231c3eb813,0,0.047622
000686d7-f4fc-448d-97a0-44fa9c5d3aa6,0,0.009341
000e3a7d-c0ca-4349-bb26-5af2d8993c3d,0,0.005463
00100a24-854d-423d-a092-edcf6179e061,0,0.152223


In [46]:
INFINITY = 100  # No regularization
lr = LogisticRegression(C=INFINITY)

In [47]:
test_chex = tc0.rename(columns={'targetPredProba':'pg0'}).drop(['targetPred'],axis=1)
test_chex = test_chex.join(tc1.rename(columns={'targetPredProba':'pg1'}).drop(['targetPred'],axis=1))
test_chex = test_chex.join(tc2.rename(columns={'targetPredProba':'pg2'}).drop(['targetPred'],axis=1))
test_chex = test_chex.join(tc3.rename(columns={'targetPredProba':'pg3'}).drop(['targetPred'],axis=1))
test_chex = test_chex.join(tc4.rename(columns={'targetPredProba':'pg4'}).drop(['targetPred'],axis=1))
print( test_chex.shape )
test_chex.head()

(3000, 5)


,pg0,pg1,pg2,pg3,pg4
patientId,,,,,
0000a175-0e68-4ca4-b1af-167204a7e0bc,0.000299,0.000348,0.000074,0.004830,0.000121
0005d3cc-3c3f-40b9-93c3-46231c3eb813,0.047622,0.306961,0.034571,0.089974,0.009179
000686d7-f4fc-448d-97a0-44fa9c5d3aa6,0.009341,0.017488,0.051991,0.046703,0.013568
000e3a7d-c0ca-4349-bb26-5af2d8993c3d,0.005463,0.004486,0.007799,0.004165,0.007487
00100a24-854d-423d-a092-edcf6179e061,0.152223,0.277856,0.170742,0.185337,0.080029


In [48]:
all_test_probs = test_prob_df.join(test_den.join(test_chex))
all_test_probs.head()

,ph0,ph1,ph2,ph3,ph4,pa0,pa1,pa2,pa3,pa4,pg0,pg1,pg2,pg3,pg4
patientId,,,,,,,,,,,,,,,
30451f0f-6692-4633-b234-94ae2b5960de,0.3098,0.3925,0.232100,0.2020,0.3391,0.431048,0.273022,0.438814,0.520895,0.300254,0.168970,0.462986,0.209652,0.350541,0.446571
28debac6-3823-4e02-bc83-3fbb9ded1718,0.3363,0.4108,0.229700,0.3239,0.4751,0.518329,0.513909,0.584034,0.645983,0.653511,0.402182,0.463429,0.558107,0.451014,0.495412
c0ec3097-a14b-4ee2-9547-c0d2532f2e62,0.2473,0.4535,0.216700,0.2501,0.3050,0.014758,0.035855,0.071825,0.124589,0.057309,0.007691,0.016391,0.068008,0.024553,0.010087
2e5ab719-7447-4d3f-95f8-5ed5c8c736a1,0.2704,0.4160,0.216700,0.4212,0.4334,0.002578,0.001543,0.015986,0.014047,0.006846,0.001181,0.001188,0.000243,0.001691,0.001115
1d0ce799-81bb-40be-8e92-4b1783906cc9,0.2484,0.0748,0.019257,0.0867,0.1869,0.431875,0.400963,0.341952,0.255228,0.374795,0.308078,0.436136,0.262940,0.422991,0.447343


In [49]:
test_sets = []
test_set = ['pa','pg','ph']
for i in range(5):
    this_set = [n + str(i) for n in test_set]
    test_sets.append(this_set)
test_sets

[['pa0', 'pg0', 'ph0'],
 ['pa1', 'pg1', 'ph1'],
 ['pa2', 'pg2', 'ph2'],
 ['pa3', 'pg3', 'ph3'],
 ['pa4', 'pg4', 'ph4']]

In [50]:
yps = []
MAXCONF = .996
Xtr = df.drop('actual',axis=1).copy()
ytr = df.actual.copy()
X_train_adjust = Xtr
X_train_adjust.loc[Xtr.ph>MAXCONF,'ph'] = MAXCONF
X_train_logit = X_train_adjust.apply(logit)
lr.fit(X_train_logit, ytr)
for tset in test_sets:
    Xte = all_test_probs[tset].copy()
    Xte.columns = test_set
    X_test_adjust = Xte
    X_test_adjust.loc[Xte.ph>MAXCONF,'ph'] = MAXCONF
    X_test_logit = X_test_adjust.apply(logit)
    yp = lr.predict_proba(X_test_logit)[:,1]
    yps.append(yp)
len(yps), [len(y) for y in yps]

(5, [3000, 3000, 3000, 3000, 3000])

In [51]:
p = pd.DataFrame(yps,columns=Xte.index).transpose().apply(logit).mean(axis=1).apply(expit)
class_preds = pd.DataFrame(p,columns=['prob'])
class_preds.index.name = 'patientId'
print(class_preds.shape)
class_preds.head()

(3000, 1)


,prob
patientId,
30451f0f-6692-4633-b234-94ae2b5960de,0.352096
28debac6-3823-4e02-bc83-3fbb9ded1718,0.508364
c0ec3097-a14b-4ee2-9547-c0d2532f2e62,0.037769
2e5ab719-7447-4d3f-95f8-5ed5c8c736a1,0.003476
1d0ce799-81bb-40be-8e92-4b1783906cc9,0.333004


In [52]:
phase4_test_output = class_preds

In [53]:
yps = []
ids = []
MAXCONF = .995
folds = [p0,p1,p2,p3,p4]
for i in range(len(folds)):
    fs = folds.copy()
    te = fs.pop(i)
    tr = np.concatenate(fs)
    Xtr = df.loc[tr,:].drop('actual',axis=1).copy()
    Xte = df.loc[te,:].drop('actual',axis=1).copy()
    ytr = df.actual[tr].copy()
    yte = df.actual[te].copy()
    X_train_adjust = Xtr
    X_test_adjust = Xte
    X_train_adjust.loc[Xtr.ph>MAXCONF,'ph'] = MAXCONF
    X_test_adjust.loc[Xte.ph>MAXCONF,'ph'] = MAXCONF
    X_train_logit = X_train_adjust.apply(logit)
    X_test_logit = X_test_adjust.apply(logit)
    lr.fit(X_train_logit, ytr)
    yp = lr.predict_proba(X_test_logit)[:,1]
    yps = yps + list(yp)
    ids = ids + list(Xte.index.values)

In [54]:
oof_class_preds = pd.DataFrame({'prob':yps},index=ids)
oof_class_preds.index.name = 'patientId'
print(oof_class_preds.shape)
oof_class_preds.head()

(25684, 1)


,prob
patientId,
00436515-870c-4b36-a041-de91049b9ab4,0.449031
00704310-78a8-4b38-8475-49f4573b2dbb,0.598099
00aecb01-a116-45a2-956c-08d2fa55433f,0.412871
01027bc3-dc40-4165-a6c3-d6be2cb7ca34,0.006151
6bde463a-d52c-4730-9833-091fc1fef528,0.006869


In [55]:
phase4_oof_output = oof_class_preds

In [56]:
df1 = phase2_output
df2 = pd.read_csv(UNET_RESULTS_PATH).rename(columns={
    'predictionString':'PredictionString'}).set_index('patientId')

class_preds = phase4_test_output
class_probs = class_preds.prob.to_dict()

df1_cases = df1[~df1.PredictionString.isnull()]
df2_cases = df2[~df2.PredictionString.isnull()]

df1_pos_ids = df1_cases.index.values
df2_pos_ids = df2_cases.index.values

In [57]:
df1_only_dict = {}
for p in df1_pos_ids:
    if not p in df2_pos_ids:
        df1_only_dict[p] = float(df1_cases.loc[p,'PredictionString'].split(' ')[0])

df2_only_dict = {}
for p in df2_pos_ids:
    if not p in df1_pos_ids:
        df2_only_dict[p] = float(df2_cases.loc[p,'PredictionString'].split(' ')[0])
        
candidates = pd.DataFrame(pd.Series(df2_only_dict,name='conf')).join(class_preds,how='left')
accepted = candidates[ (candidates.prob>UNET_MINPROB) & (candidates.conf>UNET_MINCONF)
                     ].index.values

In [58]:
accepted

array(['017a4e3a-191b-4013-a228-dabb749a3d70',
       '05a43f7e-f0c9-40b0-b705-8753feac0e26',
       '0ffb34e5-f3c7-49c9-b9cd-5b73354f562d',
       '1263a9b3-0492-402e-a152-42a024955a68',
       '143cc465-80b8-484c-ba71-ad362ea8fd90',
       '1aac71f9-fc15-4a9c-9b0f-b432f79ab2ef',
       '1b560be4-be73-4976-8d09-06f2dc181701',
       '1c5b8f7b-d76d-4a2c-ba14-42325e4e4a39',
       '1dd0b293-1563-4cf3-ae1e-fcd4232f852c',
       '21271204-1954-43bc-973b-63b896c17374',
       '215d4b0a-b546-4eae-ab7a-39b303f7248b',
       '239cc56f-29a2-4562-acaf-3282e639591c',
       '268833a7-38e7-4c97-a4f1-db5175ea34ae',
       '26b59bc8-dd04-4093-876f-fb0667d3b43a',
       '271afcc2-a753-43cc-8560-2e4d246b1502',
       '295fdc53-a88f-4fdd-8392-15c8776cde93',
       '2a9124dc-a484-4423-a61a-c7ac12d131d7',
       '2ae90a2c-813a-45f4-b350-03814995e418',
       '2b419ca4-0ce1-4eba-a59f-dfc24b74aabf',
       '2be12301-eb2a-409d-8a84-92bf849c3247',
       '2dbac7d7-9100-4861-8e15-fa2adf82c6e3',
       '2ff9e

In [59]:
df_out = df1.copy()
for p,r in df1.iterrows():
    if p in accepted:
        df_out.loc[p,'PredictionString'] = df2.loc[p,'PredictionString']
phase5_output = df_out
phase5_output.head()

,PredictionString
patientId,
c103f115-266b-4f0c-97d0-082dc2438a27,0.97 612 372 268 416 0.97 240 568 188 184
2ba3df76-d959-414f-b466-4bc0b4f41135,0.98 188 252 224 252 0.95 588 240 228 340
1114eeb8-c035-42f4-a038-13a806a568ad,NaN
2276b258-5b47-4c7d-ab1f-5b2bae4a8fc4,NaN
23ffe96d-ef6c-4a8f-914e-12754fe689c4,NaN


In [60]:
yolo_epochs = [1500, 2000, 3200, 4100, 5300, 6700, 10000, 13700]

In [61]:
df1 = phase5_output

dfs = []
for eps in yolo_epochs:
    yolo_stem = YOLO_SUB_STEM2 if (eps in MORE_YOLO_EPOCHS) else YOLO_SUB_STEM
    dfs.append( 
        pd.read_csv(RESULTS_LOC+'/'+yolo_stem+str(eps)+'.csv').set_index('patientId'))

class_preds = phase4_test_output
class_probs = class_preds.prob.to_dict()

df1_cases = df1[~df1.PredictionString.isnull()]
df1_pos_ids = df1_cases.index.values
included = df1_pos_ids.tolist()
df_out = df1.copy()

In [62]:
while True:
    for df2 in dfs:
        df2_cases = df2[~df2.PredictionString.isnull()]
        df2_pos_ids = df2_cases.index.values
        df2_only_dict = {}
        outstring_dict = {}
        for p in df2_pos_ids:
            if not p in included:
                maxconf = 0
                boxes = []
                confs = []
                s = df2.loc[p,'PredictionString'].split(' ')
                if s[-1]=='':
                    s.pop()  # remove terminating null
                if s[0]=='':
                    s.pop(0)  # remove initial null
                if ( len(s)%5 ):
                    print( 'Bad prediction string.')
                while len(s):
                    conf = float(s.pop(0))
                    x = int(round(float(s.pop(0))))
                    y = int(round(float(s.pop(0))))
                    w = int(round(float(s.pop(0))))
                    h = int(round(float(s.pop(0))))
                    if (conf>YOLO_MINCONF):
                        boxes.append( [x,y,w,h] )
                        confs.append( conf )
                        if conf>maxconf:
                            maxconf = conf
                predictionString = ''
                if len(boxes):
                    for box, conf in zip(boxes, confs):
                        x, y, w, h = box
                        # add to predictionString
                        predictionString += '{:6.4f} '.format(conf)
                        predictionString += str(x) + ' ' + str(y) + ' ' + str(w) + ' ' + str(h) + ' '
                        df2_only_dict[p] = maxconf*class_probs[p]
                outstring_dict[p] = predictionString
        candidates = pd.DataFrame(pd.Series(df2_only_dict,name='prod'))
        best = candidates.sort_values('prod').tail(1).index.values[0]
        print(best,candidates.loc[best,'prod'])
        df_out.loc[best,'PredictionString'] = outstring_dict[best]
        included.append( best )
    minprob = class_preds.loc[included[-8:],:].prob.min()
    if minprob<MINMIN:
        break

120a451e-4b61-45e2-b417-cde8bab08d7e 0.23889583954508548
2f4dcc33-3ccc-41ca-8b33-9db707891518 0.3172502197430379
30867665-1918-4b21-b3d4-b9772a918fdb 0.23347631687361017
1f55113e-db6e-4b97-853a-7e8f7e1fe451 0.21218182938086058
138ff8b1-a35f-4187-a76b-3eaa01407591 0.2014318721116639
28072866-534c-468f-92e0-ea14b79b9421 0.14762150814877056
10d00463-dfa9-45b6-8869-9674a7773069 0.2791560296415149
1ec0a41e-2a1c-4b4d-8737-06e8373c415e 0.24176859706080525
10ae6129-dc22-412d-924d-5c15f93f5408 0.11701838474529465
0371b8f9-9f18-4f4c-9c9c-38867e373437 0.13353863474572317
134cd460-818b-4ea3-9861-0bdc82b1226c 0.1508810393742686
2c097f0d-c690-4bb2-ae19-cf7cf08a4192 0.1486738591349082
02b7f2f0-b8e0-41c8-9cfc-83c69d202c67 0.19605067193972783
2b98d69d-6e7c-4d51-af53-c3fb16301c08 0.1372700425764234
1eda3687-bdf5-469f-8fba-5000c559f48b 0.23139693416457768
260a7b23-08ed-451d-bd0a-ecbffc7c3d30 0.23538761781658588
28a52994-65a7-4fe6-aa5e-adc0fd4535e9 0.10580141452110654
25eb25e4-35a7-4357-a112-264787e32094 

In [63]:
class_preds.loc[included[-32:],:]   

,prob
patientId,
120a451e-4b61-45e2-b417-cde8bab08d7e,0.389370
2f4dcc33-3ccc-41ca-8b33-9db707891518,0.633555
30867665-1918-4b21-b3d4-b9772a918fdb,0.476660
1f55113e-db6e-4b97-853a-7e8f7e1fe451,0.385601
138ff8b1-a35f-4187-a76b-3eaa01407591,0.373010
28072866-534c-468f-92e0-ea14b79b9421,0.211147
10d00463-dfa9-45b6-8869-9674a7773069,0.343005
1ec0a41e-2a1c-4b4d-8737-06e8373c415e,0.251592
10ae6129-dc22-412d-924d-5c15f93f5408,0.382470


In [64]:
df_out.to_csv('submission.csv')
df_out.head()

,PredictionString
patientId,
c103f115-266b-4f0c-97d0-082dc2438a27,0.97 612 372 268 416 0.97 240 568 188 184
2ba3df76-d959-414f-b466-4bc0b4f41135,0.98 188 252 224 252 0.95 588 240 228 340
1114eeb8-c035-42f4-a038-13a806a568ad,NaN
2276b258-5b47-4c7d-ab1f-5b2bae4a8fc4,NaN
23ffe96d-ef6c-4a8f-914e-12754fe689c4,NaN
